In [ ]:
USE SECONDARY ROLES ALL;
select * from feedback_data;

In [ ]:
df = cell1.to_pandas()
df


**DATA DESCRIPTION**

In [ ]:
df.describe().T
print(df.info())
print(df.shape)

Checking the Null Values

In [ ]:
df.isnull().any()
#null values are = COMORBIDITIES, SIDE_EFFECTS_REPORTED, TREATMENT_END_DATE, ADHERENCE, DOCTOR_NAME
#df.isnull().sum()


In [ ]:
df.isnull().sum()


In [ ]:
df

In [ ]:
import numpy as np
df = df.replace("N/A", np.nan)

df.isnull().sum()

FILLING NULL VALUES IN "COMORBIDITIES" COLUMNS

In [ ]:
df['COMORBIDITIES'] = df['COMORBIDITIES'].fillna('Not Specified')
df = df.replace("None",'Not Specified')
df['COMORBIDITIES']


In [ ]:
df['COMORBIDITIES'].isnull().sum()


FIILLING UP 'SIDE_EFFECTS' COLUMN

In [ ]:
df = df.replace(np.nan, 'Not Specified')
df= df.replace('None Reported', 'Not Specified')
df['SIDE_EFFECTS_REPORTED']

UPDATING NULL VALUES IN DOCTOR NAME TO 'UNKNOWN DOCTOR'

In [ ]:
#df['DOCTOR_NAME'] = df['DOCTOR_NAME'].fillna('Unknown Doctor')
df['DOCTOR_NAME']= df['DOCTOR_NAME'].replace('Not Specified', 'Unknown Doctor')
df['DOCTOR_NAME']

FILLING NULL VALUES IN ADHERENCE TO 'UNKNOWN'

In [ ]:
df['ADHERENCE'] = df['ADHERENCE'].replace(['Not Specified'], np.nan)
df['ADHERENCE'] = df['ADHERENCE'].fillna('Unknown')

df['ADHERENCE']

In [ ]:
df['SIDE_EFFECTS_REPORTED'] = df['SIDE_EFFECTS_REPORTED'].replace('Not Specified', 'None Reported')


In [ ]:

#Updating All 'Not Specified' values as 'Unspecified'
df.replace('Not Specified', 'Unspecified', inplace=True)


Trimming White spaces in all the below columns

In [ ]:
#white space trimming and title case
columns_to_titlecase = [
    'PATIENT_NAME', 'COUNTRY', 'LANGUAGE', 'DRUG_NAME', 'THERAPAUTIC_AREA',
    'FEEDBACK', 'LANGUAGE_CODE', 'GENDER', 'INDICATION', 'COMORBIDITIES',
    'DIAGNOSIS', 'SIDE_EFFECTS_REPORTED', 'FEEDBACK_SOURCE', 'HEALTHCARE_PROVIDER',
    'DOCTOR_NAME', 'REGION', 'FOLLOW_UP_ACTIONS'
]

for col in columns_to_titlecase:
    df[col] = df[col].astype(str).str.strip().str.title()


Converting varchar to Date

In [ ]:
date_columns = [
    'DOB',
    'FEEDBACK_DATE',
    'TREATMENT_START_DATE',
    'TREATMENT_END_DATE'
]

for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')


**FEATURE ENGINEERING**

ADDING NEW COLUMNS FEEDBACK_COUNT AND AGE


In [ ]:
#feature engineering, Feedback count based on ID, to track how many times the same patients 
#provided review
df['FEEDBACK_COUNT'] = df.groupby('ID')['FEEDBACK'].transform('count')

#Age at Feedback
df['AGE_AT_FEEDBACK'] = (
    (df['FEEDBACK_DATE'] - df['DOB']).dt.days // 365
)
# removing negative ages and above 100 ages
df = df[df['AGE_AT_FEEDBACK'].between(0, 100)]


FILLING NULL VALUES IN TREATMENT_END_DATE

CALCULATING TREATMENT DURATION DAYS 

In [ ]:
#Treatment Duration days
#Impute Null End Dates with Today’s Date (Assuming ongoing treatment)
today = pd.Timestamp('today').normalize()
df['TREATMENT_END_DATE_FILL'] = df['TREATMENT_END_DATE'].fillna(today)
df['TREATMENT_DURATION_DAYS'] = (df['TREATMENT_END_DATE_FILL'] - df['TREATMENT_START_DATE']).dt.days

#standardizing language_code to lower
df['LANGUAGE_CODE'] = df['LANGUAGE_CODE'].astype(str).str.strip().str.lower()

df

In [ ]:
#converting dates to string before writing to table
date_columns = [
    'DOB',
    'FEEDBACK_DATE',
    'TREATMENT_START_DATE',
    'TREATMENT_END_DATE',
    'TREATMENT_END_DATE_FILL'
]

# If you want ISO 8601 (YYYY-MM-DD) format for all:
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%Y-%m-%d')


df


In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
data = df
exp = df

session.use_database('SF_HACKATHON_DB')
session.use_schema('CRYPTO_COUGARS_SCHEMA')
df.reset_index(drop=True, inplace=True)

# The correct way to call write_pandas with a specific table name and overwrite
exp.reset_index(drop=True)
snowpark_df = session.write_pandas(
    df=exp,
    table_name='PROCESSED_FEEDBACK',
    overwrite=True
)





In [ ]:
session.table("Processed_Feedback").show()